In [1]:
! pip install transformers
! pip install datasets

     |████████████████████████████████| 2.8 MB 7.7 MB/s 
     |████████████████████████████████| 3.3 MB 59.0 MB/s 
     |████████████████████████████████| 50 kB 8.5 MB/s 
     |████████████████████████████████| 895 kB 62.8 MB/s 
     |████████████████████████████████| 636 kB 48.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 8.6 MB/s 
     |████████████████████████████████| 119 kB 70.7 MB/s 
     |████████████████████████████████| 243 kB 66.1 MB/s 


In [2]:
cd '/content/drive/MyDrive/CAMemBERT2'

/content/drive/MyDrive/CAMemBERT2


In [3]:
from transformers import TrainingArguments,Trainer,AutoModelForSequenceClassification,AutoModel
import numpy as np
from math import floor
import torch
from torch.cuda import is_available
from collections import defaultdict
from scipy.stats import spearmanr
from sklearn.metrics import cohen_kappa_score,classification_report,fbeta_score
from models_f import MultiTaskModel
from trainer_f import MultiTaskModelTrainer
from trainer_utils import (training_args,training_kwargs,get_dataset,
                           freeze_layers,normalise_scores,create_params_grid)

In [4]:
pretrained_model_name = 'distilroberta-base'
model_type = 'roberta' if 'roberta' in pretrained_model_name else 'bert'
model_n_layers = 6 if 'roberta' in pretrained_model_name else 12

In [10]:
dataset_obj,dataset_dict = get_dataset(['aes','ged','ner'],pretrained_model=pretrained_model_name,max_length=512)
dataset_dict = dataset_dict.map(normalise_scores)

trainer_kwargs_multi_task,model_kwargs_multi_task = training_kwargs(
    tasks=['aes','ged','ner'],
    class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
    early_stopping_patience=2,
    shared_encoder_n_layers=0,
    output_layer = {'aes':-1,
                    'ged':-1,
                    'ner':-1},
    early_stopping_metric='loss',
    normalised_values=[40,0],
    frozen_layers=None,
    save_results=True)

args = training_args(lr=4e-5,epochs=10)

for _ in range(5):

    # mutlti_taskser
    model = MultiTaskModel(kwargs_dict = model_kwargs_multi_task,
                           pretrained_model=pretrained_model_name)
    model = freeze_layers(model,frozen_layers=None,model_type=model_type)
    trainer = MultiTaskModelTrainer(
        model,
        args,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['dev'],
    )
    trainer.kwargs_dict = trainer_kwargs_multi_task
    trainer.test_dataset = dataset_dict['test']
    trainer.train()

dataset_obj,dataset_dict = get_dataset(['aes'],pretrained_model=pretrained_model_name,max_length=512)
dataset_dict = dataset_dict.map(normalise_scores)

trainer_kwargs_aes,model_kwargs_aes = training_kwargs(
  tasks=['aes'],
  class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
  early_stopping_patience=2,
  normalised_values=[40,0],
  early_stopping_metric='loss',
  frozen_layers=0,
  save_results=True)

for _ in range(5):

    # mutlti_taskser
    model = MultiTaskModel(kwargs_dict = model_kwargs_aes,
                           pretrained_model=pretrained_model_name)
    model = freeze_layers(model,frozen_layers=None,model_type=model_type)
    trainer = MultiTaskModelTrainer(
        model,
        args,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['dev'],
    )
    trainer.kwargs_dict = trainer_kwargs_aes
    trainer.test_dataset = dataset_dict['test']
    trainer.train()




Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 1,
  "use

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/97 [00:00<?, ?ex/s]

  0%|          | 0/79 [00:00<?, ?ex/s]

  0%|          | 0/1057 [00:00<?, ?ex/s]

...

aes_weight_coef : 1.0

losses {'aes': 0.014692074334662821, 'total_loss': 0.014692074334662821}

aes_metrics {'rmse_aes': 4.632029382736215, 'pearson_aes': 0.6114141270179188, 'spearman_aes': 0.6339149369278139, 'kappa_aes': 0.4857955213008436}

epochs since best performance 0

aes_weight_coef : 1.0

losses {'aes': 0.02626913061572446, 'total_loss': 0.02626913061572446}

aes_metrics {'rmse_aes': 6.09211983226142, 'pearson_aes': 0.6785346766174106, 'spearman_aes': 0.6723407726778269, 'kappa_aes': 0.3654325547040901}

epochs since best performance 0

aes_weight_coef : 1.0

losses {'aes': 0.013173190074869327, 'total_loss': 0.013173190074869327}

aes_metrics {'rmse_aes': 4.3326841448153335, 'pearson_aes': 0.6524680198659438, 'spearman_aes': 0.6629472755564925, 'kappa_aes': 0.6132792948960534}

epochs since best performance 1

aes_weight_coef : 1.0

losses {'aes': 0.028375966681374445, 'total_loss': 0.028375966681374445}

aes_metrics {'rmse_aes': 6.385307502284771, 'pearson_aes': 0.6292877



Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "p

Step,Training Loss
500,0.022300


aes_weight_coef : 1.0

losses {'aes': 0.02163430613776048, 'total_loss': 0.02163430613776048}

aes_metrics {'rmse_aes': 5.665797898159165, 'pearson_aes': 0.6510549365995653, 'spearman_aes': 0.6324570711286784, 'kappa_aes': 0.4177565255299439}

epochs since best performance 0

aes_weight_coef : 1.0

losses {'aes': 0.016925420612096786, 'total_loss': 0.016925420612096786}

aes_metrics {'rmse_aes': 4.982246963983879, 'pearson_aes': 0.6531940601197614, 'spearman_aes': 0.6641268778513523, 'kappa_aes': 0.5137604881226572}

epochs since best performance 0

aes_weight_coef : 1.0

losses {'aes': 0.015922701658887997, 'total_loss': 0.015922701658887997}

aes_metrics {'rmse_aes': 4.81558650817546, 'pearson_aes': 0.6665403798455851, 'spearman_aes': 0.6727137291530715, 'kappa_aes': 0.5542599210080873}

epochs since best performance 0

aes_weight_coef : 1.0

losses {'aes': 0.027406627622743446, 'total_loss': 0.027406627622743446}

aes_metrics {'rmse_aes': 6.2854060420886135, 'pearson_aes': 0.6645274



Training completed. Do not forget to share your model on huggingface.co/models =)




In [12]:
dataset_obj,dataset_dict = get_dataset(['ged','ner'],pretrained_model=pretrained_model_name,max_length=512)
dataset_dict = dataset_dict.map(normalise_scores)

trainer_kwargs_multi_task_ged_ner,model_kwargs_multi_task_ged_ner = training_kwargs(
    tasks=['ged','ner'],
    class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
    early_stopping_patience=2,
    shared_encoder_n_layers=0,
    output_layer = {'aes':-1,
                    'ged':-1,
                    'ner':-1},
    task_priorities={'ged':'primary_task','ner':'aux_task'},
    early_stopping_metric='loss',
    normalised_values=[40,0],
    frozen_layers=None,
    save_results=True)


args = training_args(lr=4e-5,epochs=10)

for _ in range(5):

    # mutlti_taskser
    model = MultiTaskModel(kwargs_dict = model_kwargs_multi_task_ged_ner,
                           pretrained_model=pretrained_model_name)
    model = freeze_layers(model,frozen_layers=None,model_type=model_type)
    trainer = MultiTaskModelTrainer(
        model,
        args,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['dev'],
    )
    trainer.kwargs_dict = trainer_kwargs_multi_task_ged_ner
    trainer.test_dataset = dataset_dict['test']
    trainer.train()

dataset_obj,dataset_dict = get_dataset(['ner'],pretrained_model=pretrained_model_name,max_length=512)
dataset_dict = dataset_dict.map(normalise_scores)



Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 1,
  "use

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/97 [00:00<?, ?ex/s]

  0%|          | 0/79 [00:00<?, ?ex/s]

  0%|          | 0/1057 [00:00<?, ?ex/s]

...

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/266 [00:00<?, ?ex/s]

  0%|          | 0/243 [00:00<?, ?ex/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


KeyError: ignored

In [14]:
trainer_kwargs_ner,model_kwargs_ner = training_kwargs(
  tasks=['ner'],
  task_priorities={'ner':'primary_task'},
  class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
  early_stopping_patience=2,
  shared_encoder_n_layers=0,
  output_layer = {'aes':-1,
                  'ged':-1,
                  'ner':-1},
  early_stopping_metric='loss',
  normalised_values=[40,0],
  frozen_layers=None,
  save_results=True)

args = training_args(lr=8e-5,epochs=10)

for _ in range(5):

    # mutlti_taskser
    model = MultiTaskModel(kwargs_dict = model_kwargs_ner,
                           pretrained_model=pretrained_model_name)
    model = freeze_layers(model,frozen_layers=None,model_type=model_type)
    trainer = MultiTaskModelTrainer(
        model,
        args,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['validation'],
    )
    trainer.kwargs_dict = trainer_kwargs_ner
    trainer.test_dataset = dataset_dict['test']
    trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "

Step,Training Loss
500,0.093000


ner_weight_coef : 1.0

losses {'ner': 0.055290713257184536, 'total_loss': 0.055290713257184536}

ner               precision    recall  f1-score   support

           0  0.995165133 0.996445193 0.995804751     42759
           1  0.968817204 0.978284473 0.973527823      1842
           2  0.980122324 0.980872226 0.980497132      1307
           3  0.910162003 0.921700224 0.915894776      1341
           4  0.933908046 0.865512650 0.898410504       751
           5  0.961836283 0.946652150 0.954183813      1837
           6  0.916030534 0.933852140 0.924855491       257
           7  0.875824176 0.864425163 0.870087336       922
           8  0.791907514 0.791907514 0.791907514       346

    accuracy                      0.985689810     51362
   macro avg  0.925974802 0.919961304 0.922796571     51362
weighted avg  0.985622878 0.985689810 0.985631958     51362

ner_metrics {'accuracy_ner': 0.9856898095868541, 'f1_score_macro_ner': 0.9227965713368103, 'f1_score_weighted_ner': 0.98563195



Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LAB

Step,Training Loss
500,0.089300
1000,0.008800


...

ner_weight_coef : 1.0

losses {'ner': 0.0560206723484126, 'total_loss': 0.0560206723484126}

ner               precision    recall  f1-score   support

           0  0.995325792 0.996000842 0.995663203     42759
           1  0.979880370 0.978284473 0.979081771      1842
           2  0.986943164 0.983167559 0.985051744      1307
           3  0.875524476 0.933631618 0.903644894      1341
           4  0.917712692 0.876165113 0.896457766       751
           5  0.965129359 0.934131737 0.949377593      1837
           6  0.884328358 0.922178988 0.902857143       257
           7  0.855135135 0.857917570 0.856524093       922
           8  0.806349206 0.734104046 0.768532526       346

    accuracy                      0.984833145     51362
   macro avg  0.918480950 0.912842439 0.915243415     51362
weighted avg  0.984870856 0.984833145 0.984797907     51362

ner_metrics {'accuracy_ner': 0.9848331451267474, 'f1_score_macro_ner': 0.9152434147699883, 'f1_score_weighted_ner': 0.984797907257



Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LAB

Step,Training Loss
500,0.093200
1000,0.009400


ner_weight_coef : 1.0

losses {'ner': 0.0560206723484126, 'total_loss': 0.0560206723484126}

ner               precision    recall  f1-score   support

           0  0.995325792 0.996000842 0.995663203     42759
           1  0.979880370 0.978284473 0.979081771      1842
           2  0.986943164 0.983167559 0.985051744      1307
           3  0.875524476 0.933631618 0.903644894      1341
           4  0.917712692 0.876165113 0.896457766       751
           5  0.965129359 0.934131737 0.949377593      1837
           6  0.884328358 0.922178988 0.902857143       257
           7  0.855135135 0.857917570 0.856524093       922
           8  0.806349206 0.734104046 0.768532526       346

    accuracy                      0.984833145     51362
   macro avg  0.918480950 0.912842439 0.915243415     51362
weighted avg  0.984870856 0.984833145 0.984797907     51362

ner_metrics {'accuracy_ner': 0.9848331451267474, 'f1_score_macro_ner': 0.9152434147699883, 'f1_score_weighted_ner': 0.984797907257



Training completed. Do not forget to share your model on huggingface.co/models =)




In [15]:
dataset_obj,dataset_dict = get_dataset(['aes'],pretrained_model=pretrained_model_name,max_length=512)
dataset_dict = dataset_dict.map(normalise_scores)

trainer_kwargs_aes,model_kwargs_aes = training_kwargs(
  tasks=['aes'],
  class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
  early_stopping_patience=2,
  normalised_values=[40,0],
  early_stopping_metric='loss',
  frozen_layers=0,
  save_results=True)

for _ in range(5):

    # mutlti_taskser
    model = MultiTaskModel(kwargs_dict = model_kwargs_aes,
                           pretrained_model=pretrained_model_name)
    model = freeze_layers(model,frozen_layers=None,model_type=model_type)
    trainer = MultiTaskModelTrainer(
        model,
        args,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['dev'],
    )
    trainer.kwargs_dict = trainer_kwargs_aes
    trainer.test_dataset = dataset_dict['test']
    trainer.train()

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 1,
  "use

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/97 [00:00<?, ?ex/s]

  0%|          | 0/79 [00:00<?, ?ex/s]

  0%|          | 0/1057 [00:00<?, ?ex/s]

...

aes_weight_coef : 1.0

losses {'aes': 0.02152849899397956, 'total_loss': 0.02152849899397956}

aes_metrics {'rmse_aes': 5.606280384250812, 'pearson_aes': 0.5438627268412554, 'spearman_aes': 0.5589993307574347, 'kappa_aes': 0.13130059501008717}

epochs since lowest loss 0

aes_weight_coef : 1.0

losses {'aes': 0.02786002918663952, 'total_loss': 0.02786002918663952}

aes_metrics {'rmse_aes': 6.3902615782121694, 'pearson_aes': 0.5364155494823838, 'spearman_aes': 0.56752792104888, 'kappa_aes': 0.24590634325503102}

epochs since lowest loss 1

aes_weight_coef : 1.0

losses {'aes': 0.01431475905701518, 'total_loss': 0.01431475905701518}

aes_metrics {'rmse_aes': 4.599119343328375, 'pearson_aes': 0.5938354229017427, 'spearman_aes': 0.6203798370963548, 'kappa_aes': 0.5735695298705834}

epochs since lowest loss 0

aes_weight_coef : 1.0

losses {'aes': 0.023453102861013677, 'total_loss': 0.023453102861013677}

aes_metrics {'rmse_aes': 5.868819129491344, 'pearson_aes': 0.5936603134132629, 'spearm



Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "p

Step,Training Loss
500,0.024200


aes_weight_coef : 1.0

losses {'aes': 0.02152849899397956, 'total_loss': 0.02152849899397956}

aes_metrics {'rmse_aes': 5.606280384250812, 'pearson_aes': 0.5438627268412554, 'spearman_aes': 0.5589993307574347, 'kappa_aes': 0.13130059501008717}

epochs since lowest loss 0

aes_weight_coef : 1.0

losses {'aes': 0.02786002918663952, 'total_loss': 0.02786002918663952}

aes_metrics {'rmse_aes': 6.3902615782121694, 'pearson_aes': 0.5364155494823838, 'spearman_aes': 0.56752792104888, 'kappa_aes': 0.24590634325503102}

epochs since lowest loss 1

aes_weight_coef : 1.0

losses {'aes': 0.01431475905701518, 'total_loss': 0.01431475905701518}

aes_metrics {'rmse_aes': 4.599119343328375, 'pearson_aes': 0.5938354229017427, 'spearman_aes': 0.6203798370963548, 'kappa_aes': 0.5735695298705834}

epochs since lowest loss 0

aes_weight_coef : 1.0

losses {'aes': 0.023453102861013677, 'total_loss': 0.023453102861013677}

aes_metrics {'rmse_aes': 5.868819129491344, 'pearson_aes': 0.5936603134132629, 'spearm



Training completed. Do not forget to share your model on huggingface.co/models =)




In [16]:
dataset_obj,dataset_dict = get_dataset(['aes','ged'],pretrained_model=pretrained_model_name,max_length=512)
dataset_dict = dataset_dict.map(normalise_scores)

trainer_kwargs_aes,model_kwargs_aes = training_kwargs(
  tasks=['aes','ged'],
  class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
  early_stopping_patience=2,
  normalised_values=[40,0],
  early_stopping_metric='loss',
  frozen_layers=0,
  save_results=True)

for _ in range(5):

    # mutlti_taskser
    model = MultiTaskModel(kwargs_dict = model_kwargs_aes,
                           pretrained_model=pretrained_model_name)
    model = freeze_layers(model,frozen_layers=None,model_type=model_type)
    trainer = MultiTaskModelTrainer(
        model,
        args,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['dev'],
    )
    trainer.kwargs_dict = trainer_kwargs_aes
    trainer.test_dataset = dataset_dict['test']
    trainer.train()

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 1,
  "use

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/97 [00:00<?, ?ex/s]

  0%|          | 0/79 [00:00<?, ?ex/s]

  0%|          | 0/1057 [00:00<?, ?ex/s]

...

aes_weight_coef : 1.0
ged_weight_coef : 1.0

losses {'aes': 0.014017063917385207, 'ged': 0.5194596648216248, 'total_loss': 0.53347672873901}

aes_metrics {'rmse_aes': 4.546524615126302, 'pearson_aes': 0.62165157802374, 'spearman_aes': 0.6318720119591705, 'kappa_aes': 0.5684600521578707}

ged               precision    recall  f1-score   support

           0       0.96      0.77      0.85     27581
           1       0.35      0.78      0.48      4305

    accuracy                           0.77     31886
   macro avg       0.65      0.78      0.67     31886
weighted avg       0.88      0.77      0.80     31886

ged_metrics {'accuracy_ged': 0.7734115285705325, 'f1_score_macro_ged': 0.668992227796267, 'f1_score_weighted_ged': 0.8047040846739153, 'f_0_5_ged': 0.6276959690614309}

epochs since lowest loss 0

aes_weight_coef : 1.0
ged_weight_coef : 1.0

losses {'aes': 0.01314904209640291, 'ged': 0.5064834190739526, 'total_loss': 0.5196324611703554}

aes_metrics {'rmse_aes': 4.3312231148817



Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "p

Step,Training Loss
500,0.458800


aes_weight_coef : 1.0
ged_weight_coef : 1.0

losses {'aes': 0.014017063917385207, 'ged': 0.5194596648216248, 'total_loss': 0.53347672873901}

aes_metrics {'rmse_aes': 4.546524615126302, 'pearson_aes': 0.62165157802374, 'spearman_aes': 0.6318720119591705, 'kappa_aes': 0.5684600521578707}

ged               precision    recall  f1-score   support

           0       0.96      0.77      0.85     27581
           1       0.35      0.78      0.48      4305

    accuracy                           0.77     31886
   macro avg       0.65      0.78      0.67     31886
weighted avg       0.88      0.77      0.80     31886

ged_metrics {'accuracy_ged': 0.7734115285705325, 'f1_score_macro_ged': 0.668992227796267, 'f1_score_weighted_ged': 0.8047040846739153, 'f_0_5_ged': 0.6276959690614309}

epochs since lowest loss 0

aes_weight_coef : 1.0
ged_weight_coef : 1.0

losses {'aes': 0.01314904209640291, 'ged': 0.5064834190739526, 'total_loss': 0.5196324611703554}

aes_metrics {'rmse_aes': 4.3312231148817



Training completed. Do not forget to share your model on huggingface.co/models =)




In [17]:
dataset_obj,dataset_dict = get_dataset(['aes','ner'],pretrained_model=pretrained_model_name,max_length=512)
dataset_dict = dataset_dict.map(normalise_scores)

trainer_kwargs_aes,model_kwargs_aes = training_kwargs(
  tasks=['aes','ner'],
  class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
  early_stopping_patience=2,
  normalised_values=[40,0],
  early_stopping_metric='loss',
  frozen_layers=0,
  save_results=True)

for _ in range(5):

    # mutlti_taskser
    model = MultiTaskModel(kwargs_dict = model_kwargs_aes,
                           pretrained_model=pretrained_model_name)
    model = freeze_layers(model,frozen_layers=None,model_type=model_type)
    trainer = MultiTaskModelTrainer(
        model,
        args,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['dev'],
    )
    trainer.kwargs_dict = trainer_kwargs_aes
    trainer.test_dataset = dataset_dict['test']
    trainer.train()

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 1,
  "use

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/97 [00:00<?, ?ex/s]

  0%|          | 0/79 [00:00<?, ?ex/s]

  0%|          | 0/1057 [00:00<?, ?ex/s]

...

aes_weight_coef : 1.0
ner_weight_coef : 1.0

losses {'aes': 0.003638013081928325, 'ner': 0.04662439322402311, 'total_loss': 0.05026240630595143}

aes_metrics {'rmse_aes': 5.287564026405102, 'pearson_aes': 0.5493229904060827, 'spearman_aes': 0.5676067012791974, 'kappa_aes': 0.2530593034201445}

ner               precision    recall  f1-score   support

           0  0.997326830 0.994691176 0.996007260     42759
           1  0.989479513 0.970141151 0.979714912      1842
           2  0.975775927 0.986228003 0.980974125      1307
           3  0.874647887 0.926174497 0.899674031      1341
           4  0.940493469 0.862849534 0.900000000       751
           5  0.978848414 0.906913446 0.941508901      1837
           6  0.935483871 0.902723735 0.918811881       257
           7  0.779505947 0.924078091 0.845657568       922
           8  0.693363844 0.875722543 0.773946360       346

    accuracy                      0.984210116     51362
   macro avg  0.907213967 0.927724686 0.915143893



Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "p

Step,Training Loss
500,0.084100
1000,0.022900
1500,0.013000
2000,0.008600
2500,0.006000


aes_weight_coef : 1.0
ner_weight_coef : 1.0

losses {'aes': 0.005114711864396583, 'ner': 0.04870338599349177, 'total_loss': 0.05381809785788835}

aes_metrics {'rmse_aes': 6.295942255841368, 'pearson_aes': 0.4779103570607161, 'spearman_aes': 0.4865118724126992, 'kappa_aes': 0.07372105378293092}

ner               precision    recall  f1-score   support

           0  0.997096087 0.995743586 0.996419377     42759
           1  0.988313856 0.964169381 0.976092333      1842
           2  0.983895706 0.981637337 0.982765224      1307
           3  0.838225256 0.915734526 0.875267284      1341
           4  0.902777778 0.865512650 0.883752549       751
           5  0.991260924 0.864452912 0.923524280      1837
           6  0.920000000 0.805447471 0.858921162       257
           7  0.790654206 0.917570499 0.849397590       922
           8  0.633891213 0.875722543 0.735436893       346

    accuracy                      0.982399439     51362
   macro avg  0.894012781 0.909554545 0.89795296



Training completed. Do not forget to share your model on huggingface.co/models =)


